In [ ]:
library(dplyr)
library(tsibble)
library(fable)
library(ggplot2)
library(tidyr)
library(purrr)
library(feasts)
library(patchwork)
options(repr.plot.width = 20, repr.plot.height = 7, repr.plot.res = 100)

# Source Helper Functions
source("../Baseline/baseline_helpers.R")
source("../Data_Inspection/data_cleaning_helpers.R")

# Clean validation data
validation <- get_validation_data()

dates <- get_dates()

train <- get_train_data()


In [ ]:
train |>
  filter(product == "FOODS_3_001") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_002") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_003") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
train |>
  filter(product == "FOODS_3_004") |>
  model(
    STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  ) |>
  components() |>
  autoplot()
# train |> filter(product == "FOODS_3_001") |> features(sales, feat_acf)
# train |>  features(sales, unitroot_ndiffs)
# train |>  features(sales, feat_acf)


In [ ]:
feat_stl <- train |>
  group_by(product) |>
  features(
    sales,
    feat_stl,
    .model = STL(
      sales ~ trend(window = 365) +
        season(period = 7) +
        season(period = 365),
      robust = TRUE
    )
  )


In [ ]:
intermittency_features <- function(y) {
  y <- as.numeric(y)
  y[is.na(y)] <- 0

  nz_idx <- which(y > 0)
  p_zero <- mean(y == 0)

  # Average inter-demand interval (ADI)
  adi <- if (length(nz_idx) <= 1) Inf else mean(diff(nz_idx))

  # Non-zero statistics
  y_nz <- y[y > 0]
  mean_nz <- if (length(y_nz) == 0) 0 else mean(y_nz)
  var_nz <- if (length(y_nz) <= 1) 0 else var(y_nz)

  cv2 <- if (mean_nz <= 0) Inf else var_nz / (mean_nz^2)

  tibble(
    p_zero = p_zero,
    adi = adi,
    mean_nz = mean_nz,
    cv2 = cv2
  )
}

feat_int <- train |>
  group_by(product) |>
  summarise(intermittency_features(sales), .groups = "drop")
feat_int


In [ ]:
feat_int$adi


In [ ]:
features_all <- feat_int |>
  left_join(stl_feats, by = "product")


In [ ]:
classify_sba <- function(adi, cv2) {
  if (!is.finite(adi) || is.infinite(adi) || !is.finite(cv2)) {
    return("dead_or_insufficient")
  }
  if (adi < 1.32 && cv2 < 0.49) {
    return("smooth")
  }
  if (adi >= 1.32 && cv2 < 0.49) {
    return("intermittent")
  }
  if (adi < 1.32 && cv2 >= 0.49) {
    return("erratic")
  }
  return("lumpy")
}

features_all <- features_all |>
  mutate(class = mapply(classify_sba, adi, cv2))
